# Parsing PDFs Homework

With the power of pdfminer, pytesseract, Camelot, and Tika, let's analyze some documents!

> If at any point you think, **"I'm close enough, I'd just edit the rest of it in Excel"**: that's fine! Just make a note of it.

## A trick to use again and again

### Approach 1

Before we get started: when you want to take the first row of your data and set it as the header, use this trick.

In [1]:
import pandas as pd
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


In [2]:
# Set the first column as the columns
df.columns = df.loc[0]

# Drop the first row
df = df.drop(0)

df

,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


🚀 Done!

### Approach 2

Another alternative is to use `.rename` on your columns and just filter out the columns you aren't interested in. This can be useful if the column name shows up multiple times in your data for some reason or another.

In [3]:
# Starting with the same-ish data...
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'fruit name', 'likes' ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,fruit name,likes
4,sweet potato,45
5,peach,12


In [4]:
df = df.rename(columns={
    0: 'fruit name',
    1: 'likes'
})
df = df[df['fruit name'] != 'fruit name']
df

,fruit name,likes
1,apple,15
2,carrot,3
4,sweet potato,45
5,peach,12


🚀 Done!

### Useful tips about coordinates

If you want to grab only a section of the page [Kull](https://jsoma.github.io/kull/#/) might be helpful in finding the coordinates.

> **Alternatively** run `%matplotlib notebook` in a cell. Afterwards, every time you use something like `camelot.plot(tables[0]).show()` it will get you nice zoomable, hoverable versions that include `x` and `y` coordinates as you move your mouse.

Coordinates are given as `"left_x,top_y,right_x,bottom_y"` with `(0,0)` being in the bottom left-hand corner.

Note that all coordinates are strings, for some reason. It won't be `[1, 2, 3, 4]` it will be `['1,2,3,4']`

# The homework

This is **mostly Camelot work**, because I don't really have any good image-based PDFs to stretch your wings on tesseract. If you know of any, let me know and I can put together another couple exercises.

## Prison Inmates

Working from [InmateList.pdf](InmateList.pdf), save a CSV file that includes every inmate.

* Make sure your rows are *all data*, and you don't have any people named "Inmate Name."


In [5]:
import tika
from tika import parser

In [6]:
inmates = parser.from_file('InmateList.pdf')
inmates_df = pd.DataFrame(columns=inmates['content'].strip().split('\n')[0].split(' '))
inmates_df = inmates_df.drop(columns=['#','Name','Date'])
inmates_df = inmates_df.rename(columns={'ICN': 'ICN #', 'Inmate': 'Inmate Name', 'Booking': 'Booking Date'})
inmates_df

,ICN #,Inmate Name,Facility,Booking Date


In [7]:
inmates_ls = inmates['content'].strip()[87:].split('\n\n')

In [8]:
inmates_ls

['70693 ABDALLAH, MICHAEL ECHC 04/30/2021',
 '152645 ABDI, ABDI ECCF 06/20/2021',
 '144666 ABDULLAH, DHAFIR ECCF 06/17/2021',
 '156374 ACEVEDO, CARLOS ECHC 06/06/2021',
 '57243 ACKER, RAYMOND P ECCF 11/02/2020',
 '68579 ADAMS, JERMAIN C ECHC 09/19/2019',
 '45262 ADAMS, MARQUIS ECHC 05/27/2021',
 '75738 AKRIGHT, JOSEPH A ECCF 05/29/2021',
 '104048 ALBERTSON, ANDREW ECCF 12/04/2019',
 '1577 ALEXANDER, BRIAN ECHC 07/13/2021',
 '148951 ALICEA, PABLO E ECCF 05/19/2020',
 '139806 ALKULIFI, ABDULLAH ECHC 07/01/2021',
 '124661 ALLEN, DARNELLE ECCF 12/03/2020',
 '125234 ALLEN, DEMONDE C ECHC 05/03/2021',
 '131437 ALLEN, GLENN A ECCF 06/12/2021',
 '144648 ALLEN, KASSEEN F ECHC 06/29/2021',
 '155564 ALLEN, TIFFANY M ECHC 04/12/2021',
 '94012 AMAN, JOSHUA D ECHC 06/27/2021',
 '60170 ANDREWS, NICOLE J ECHC 04/24/2021',
 '1550 ARCARA, SHAWN ECCF 01/20/2021',
 '69158 ARCHER, MARTELL ECHC 07/01/2021',
 '84791 AULS, RAYMOND ECHC 06/03/2021',
 '124759 AUSTIN, RASHAWN G ECHC 01/28/2021',
 '150703 AYALA, 

In [9]:
# pandas series and dataframes vs. python list vs. numpy arrays
inmates_pd = pd.Series(inmates_ls)
re_keep = r'^[0-9]+ [A-Z]+, [A-Z]+'
inmates_pd = inmates_pd[inmates_pd.str.contains(re_keep,regex=True)]

In [10]:
inmates_df

,ICN #,Inmate Name,Facility,Booking Date


In [11]:
inmates_df['Facility'] = inmates_pd.str.rsplit(n=2, expand=True)[1]
inmates_df['Booking Date'] = inmates_pd.str.rsplit(n=2, expand=True)[2]
inmates_df['ICN #'] = inmates_pd.str.split(n=1, expand=True)[0]
inmates_df

,ICN #,Inmate Name,Facility,Booking Date
0,70693,NaN,ECHC,04/30/2021
1,152645,NaN,ECCF,06/20/2021
2,144666,NaN,ECCF,06/17/2021
3,156374,NaN,ECHC,06/06/2021
4,57243,NaN,ECCF,11/02/2020
...,...,...,...,...
754,18303,NaN,ECHC,05/24/2021
755,155109,NaN,ECHC,06/27/2021
756,63815,NaN,ECHC,07/06/2021
757,20740,NaN,ECCF,11/04/2020


In [12]:
#inmates_df['Inmate Name'].str.rsplit(n=2, expand=True)[0]
names = inmates_pd.str.rsplit(n=2, expand=True)
inmates_df['Inmate Name'] = names[0].str.split(n=1, expand=True)[1]
inmates_df

,ICN #,Inmate Name,Facility,Booking Date
0,70693,"ABDALLAH, MICHAEL",ECHC,04/30/2021
1,152645,"ABDI, ABDI",ECCF,06/20/2021
2,144666,"ABDULLAH, DHAFIR",ECCF,06/17/2021
3,156374,"ACEVEDO, CARLOS",ECHC,06/06/2021
4,57243,"ACKER, RAYMOND P",ECCF,11/02/2020
...,...,...,...,...
754,18303,"ZANGHI, SANTINO J",ECHC,05/24/2021
755,155109,"ZARCONE, THOMAS E",ECHC,06/27/2021
756,63815,"ZIELINSKI, LARRY",ECHC,07/06/2021
757,20740,"ZIELINSKI, LAWRENCE T",ECCF,11/04/2020


In [13]:
inmates_df.to_csv('inmates_done.csv')

## WHO resolutions

Using [A74_R13-en.pdf](A74_R13-en.pdf), what ten member countries are given the highest assessments?

* You might need to have two separate queries, and combine the results: that last page is pretty awful!
* Always rename your columns
* Double-check that your sorting looks right......
* You can still get the answer even without perfectly clean data

In [14]:
import camelot
tables = camelot.read_pdf('A74_R13-en.pdf', flavor='stream', pages='1-6')
tables

<TableList n=6>

In [49]:
tables_pieces = [table.df.drop([0,1,2]) for table in tables[0:5]]
tables_pieces

[                                   0       1
 3                        Afghanistan  0.0070
 4                            Albania  0.0080
 5                            Algeria  0.1380
 6                            Andorra  0.0050
 7                             Angola  0.0100
 8                Antigua and Barbuda  0.0020
 9                          Argentina  0.9151
 10                           Armenia  0.0070
 11                         Australia  2.2101
 12                           Austria  0.6770
 13                        Azerbaijan  0.0490
 14                           Bahamas  0.0180
 15                           Bahrain  0.0500
 16                        Bangladesh  0.0100
 17                          Barbados  0.0070
 18                           Belarus  0.0490
 19                           Belgium  0.8211
 20                            Belize  0.0010
 21                             Benin  0.0030
 22                            Bhutan  0.0010
 23  Bolivia (Plurinational State 

In [50]:
# pd.DataFrame([tables[0:4] + last_table])
last_table = tables[5].df.drop(columns=[0,2,4]).drop([0,1,2,3,6,7,8,9])
last_table.columns = [0,1]
last_table

,0,1
4,Zambia,0.0090
5,Zimbabwe,0.0050


In [62]:
scales = pd.concat(tables_pieces, ignore_index=True).append(last_table)
scales.columns = ['Member','Percent']
scales = scales.reset_index()
scales = scales.drop(columns='index').set_index('Member')
scales.tail(10)

,Percent
Member,
United Republic of Tanzania,0.0100
United States of America,22.0000
Uruguay,0.0870
Uzbekistan,0.0320
Vanuatu,0.0010
Venezuela (Bolivarian Republic of),0.7280
Viet Nam,0.0770
Yemen,0.0100
Zambia,0.0090


In [65]:
scales['Percent'].sort_values(ascending=False).head(10)

Member
Japan                        8.5645
Germany                      6.0904
Northern Ireland             4.5673
France                       4.4273
Italy                        3.3072
United States of America    22.0000
Brazil                       2.9482
Canada                       2.7342
Russian Federation           2.4052
Republic of Korea            2.2671
Name: Percent, dtype: object

## The Avengers

Using [THE_AVENGERS.pdf](THE_AVENGERS.pdf), approximately how many lines does Captain America have as compared to Thor and Iron Man?

* Character names only: we're only counting `IRON MAN` as Iron Man, not `TONY`.
* Your new best friend might be `\n`
* Look up `.count` for strings

## COVID data

Using [covidweekly2721.pdf](covidweekly2721.pdf), what's the total number of tests performed in Minnesota? Use the Laboratory Test Rates by County of Residence chart.

* You COULD pull both tables separately OR you could pull them both at once and split them in pandas.
* Remember you can do things like `df[['name','age']]` to ask for multiple columns

## Theme Parks

Using [2019-Theme-Index-web-1.pdf](2019-Theme-Index-web-1.pdf), save a CSV of the top 10 theme park groups worldwide.

* You can clean the results or you can restrict the area the table is pulled from, up to you

## Hunting licenses

Using [US_Fish_and_Wildlife_Service_2021.pdf](US_Fish_and_Wildlife_Service_2021.pdf) and [a CSV of state populations](http://goodcsv.com/geography/us-states-territories/), find the states with the highest per-capita hunting license holders.